In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%store -r data

# 1.0 Data Cleaning Walkthrough: Combining the Data

## 1.1 Introduction

In the last mission, we began investigating possible relationships between **SAT scores** and **demographic factors**. In order to do this, we acquired several data sets about [New York City public schools](https://data.cityofnewyork.us/data?cat=education). We manipulated these data sets, and found that we could combine them all using the **DBN** column. All of the data sets are currently stored as **keys** in the **data** dictionary. Each individual data set is a pandas dataframe.

In this section, **we'll clean the data a bit more**, then **combine** it. Finally, we'll **compute correlations** and perform some analysis.

The first thing we'll need to do in preparation for the merge is condense some of the data sets. In the last section, we noticed that the values in the **DBN** column were unique in the **sat_results** data set. Other data sets like **class_size** had duplicate **DBN** values, however.

We'll need to condense these data sets so that each value in the **DBN** column is unique. If not, we'll run into issues when it comes time to combine the data sets.

While the main data set we want to analyze, **sat_results**, has unique **DBN** values for every high school in New York City, other data sets aren't as clean. A single row in the **sat_results** data set may match multiple rows in the **class_size** data set, for example. This situation will create problems, because we don't know which of the multiple entries in the **class_size** data set we should combine with the single matching entry in **sat_results**. Here's a diagram that illustrates the problem:


<left><img width="400" src="https://drive.google.com/uc?export=view&id=1deYm5RdQXO2xMX6dUgHLvqDEWipk3axq"></left>

In the diagram above, we can't just combine the rows from both data sets because there are several cases where multiple rows in **class_size** match a single row in **sat_results.**

To resolve this issue, we'll condense the **class_size**, **graduation**, and **demographics** data sets so that each **DBN** is unique.

## 1.2 Condensing the Class Size Data Set

The first data set that we'll condense is **class_size**. The first few rows of **class_size** look like this:

|__| CSD | BOROUGH | SCHOOL CODE | SCHOOL NAME               | GRADE | PROGRAM TYPE | CORE SUBJECT (MS CORE and 9-12 ONLY) | CORE COURSE (MS CORE and 9-12 ONLY) | SERVICE CATEGORY(K-9* ONLY) | NUMBER OF STUDENTS / SEATS FILLED | NUMBER OF SECTIONS |
|---|-----|---------|-------------|---------------------------|-------|--------------|--------------------------------------|-------------------------------------|-----------------------------|-----------------------------------|--------------------|
| 0 | 1   | M       | M015        | P.S. 015 Roberto Clemente | 0K    | GEN ED       | -                                    | -                                   | -                           | 19.0                              | 1.0                |
| 1 | 1   | M       | M015        | P.S. 015 Roberto Clemente | 0K    | CTT          | -                                    | -                                   | -                           | 21.0                              | 1.0                |
| 2 | 1   | M       | M015        | P.S. 015 Roberto Clemente | 01    | GEN ED       | -                                    | -                                   | -                           | 17.0                              | 1.0                |
| 3 | 1   | M       | M015        | P.S. 015 Roberto Clemente | 01    | CTT          | -                                    | -                                   | -                           | 17.0                              | 1.0                |
| 4 | 1   | M       | M015        | P.S. 015 Roberto Clemente | 02    | GEN ED       | -                                    | -                                   | -                           | 15.0                              | 1.0                |

As you can see, the first few rows all pertain to the same school, which is why the **DBN** appears more than once. It looks like each school has multiple values for **GRADE**, **PROGRAM TYPE**, **CORE SUBJECT (MS CORE and 9-12 ONLY)**, and **CORE COURSE (MS CORE and 9-12 ONLY)**.

If we look at the unique values for **GRADE**, we get the following:

```python
array(['0K', '01', '02', '03', '04', '05', '0K-09', nan, '06', '07', '08',
       'MS Core', '09-12', '09'], dtype=object)
```

Because we're dealing with high schools, we're only concerned with grades 9 through 12. That means we only want to pick rows where the value in the **GRADE** column is **09-12**.

If we look at the unique values for **PROGRAM TYPE**, we get the following:

```python
array(['GEN ED', 'CTT', 'SPEC ED', nan, 'G&T'], dtype=object)
```

Each school can have multiple program types. Because **GEN ED** is the largest category by far, let's only select rows where **PROGRAM TYPE** is **GEN ED**.



## 1.3 Condensing the Class Size Data Set

**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>


- Create a new variable called **class_size**, and assign the value of **data["class_size"]** to it.
- Filter **class_size** so the **GRADE** column only contains the value **09-12.** Note that the name of the **GRADE** column has a space at the end; you'll generate an error if you don't include it.
- Filter **lass_size** so that the **PROGRAM TYPE** column only contains the value **GEN ED.**
- Display the first five rows of **class_size** to verify.

In [133]:
# put your code here
class_size = data['class_size']
class_size = class_size[class_size['GRADE '] == '09-12']
class_size = class_size[class_size['PROGRAM TYPE'] == 'GEN ED']
class_size.head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
225,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 9,-,63.0,3.0,21.0,19.0,25.0,STARS,NaN,01,01M292
226,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 10,-,79.0,3.0,26.3,24.0,31.0,STARS,NaN,01,01M292
227,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 11,-,38.0,2.0,19.0,16.0,22.0,STARS,NaN,01,01M292
228,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 12,-,69.0,3.0,23.0,13.0,30.0,STARS,NaN,01,01M292
229,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Integrated Algebra,-,53.0,3.0,17.7,16.0,21.0,STARS,NaN,01,01M292


## 1.4 Computing Average Class Sizes

As we saw when we displayed **class_size** on the last screen, **DBN** still isn't completely unique. This is due to the **CORE COURSE (MS CORE and 9-12 ONLY)** and **CORE SUBJECT (MS CORE and 9-12 ONLY)** columns.

**CORE COURSE (MS CORE and 9-12 ONLY)** and **CORE SUBJECT (MS CORE and 9-12 ONLY)** seem to pertain to different kinds of classes. For example, here are the unique values for **CORE SUBJECT (MS CORE and 9-12 ONLY)**:

```python
array(['ENGLISH', 'MATH', 'SCIENCE', 'SOCIAL STUDIES'], dtype=object)
```

This column only seems to include certain subjects. We want our class size data to include every single class a school offers -- not just a subset of them. What we can do is take the average across all of the classes a school offers. This will give us unique **DBN** values, while also incorporating as much data as possible into the average.

Fortunately, we can use the [pandas.DataFrame.groupby()](http://pandas.pydata.org/pandas-docs/stable/groupby.html) method to help us with this. The **DataFrame.groupby()** method will split a dataframe up into unique groups, based on a given column. We can then use the **agg()** method on the resulting **pandas.core.groupby** object to find the **mean** of each column.

Let's say we have this data set:

<left><img width="500" src="https://drive.google.com/uc?export=view&id=1sJjENlTRR56RwYzBmmsU8aIMELgjx8zg"></left>

Using the **groupby()** method, we'll split this dataframe into four separate groups -- one with the **DBN 01M292**, one with the **DBN 01M332**, one with the **DBN 01M378**, and one with the **DBN 01M448**:

<left><img width="500" src="https://drive.google.com/uc?export=view&id=1y9imbMLKRDI50wQqPn7P6TAd6MfCL4Nq"></left>

<left><img width="500" src="https://drive.google.com/uc?export=view&id=1FitnyClxHDQLnoAB3jR7YI_jEPZZhkco"></left>

Then, we can compute the averages for the **AVERAGE CLASS SIZE** column in each of the four groups using the **agg()** method:

<left><img width="200" src="https://drive.google.com/uc?export=view&id=1gHVZixGOuGYYON_zU0OUPTJcC9Q_mKeV"></left>

After we group a dataframe and aggregate data based on it, the column we performed the grouping on (in this case **DBN**) will become the index, and will no longer appear as a column in the data itself. To undo this change and keep **DBN** as a column, we'll need to use [pandas.DataFrame.reset_index()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html). This method will reset the index to a list of integers and make **DBN** a column again.

## 1.5 Computing Average Class Sizes

- Find the average values for each column associated with each **DBN** in **class_size**.
    - Use the [pandas.DataFrame.groupby()](http://pandas.pydata.org/pandas-docs/stable/groupby.html) method to group **class_size** by **DBN**.
    - Use the [agg()](http://pandas.pydata.org/pandas-docs/stable/groupby.html#aggregation) method on the resulting **pandas.core.groupby** object, along with the **numpy.mean()** function as an argument, to calculate the average of each group.
    - Assign the result back to **class_size**.
- Reset the index to make **DBN** a column again.
    - Use the [pandas.DataFrame.reset_index()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html) method, along with the keyword argument **inplace=True**.
- Assign **class_size** back to the **class_size** key of the **data** dictionary.
- Display the first few rows of **data["class_size"]** to verify that everything went okay.

In [138]:
# put your code here
import numpy as np

class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace = True)
data['class_size'] = class_size
data['class_size'].head()

,DBN,CSD,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,SCHOOLWIDE PUPIL-TEACHER RATIO
0,01M292,1,88.0000,4.000000,22.564286,18.50,26.571429,NaN
1,01M332,1,46.0000,2.000000,22.000000,21.00,23.500000,NaN
2,01M378,1,33.0000,1.000000,33.000000,33.00,33.000000,NaN
3,01M448,1,105.6875,4.750000,22.231250,18.25,27.062500,NaN
4,01M450,1,57.6000,2.733333,21.200000,19.40,22.866667,NaN


## 1.6 Condensing the Demographics Data Set

Now that we've finished condensing **class_size**, let's condense **demographics**. The first few rows look like this:

| _| DBN    | Name                      | schoolyear | fl_percent | frl_percent | total_enrollment | prek | k  | grade1 | grade2 |
|---|--------|---------------------------|------------|------------|-------------|------------------|------|----|--------|--------|
| 0 | 01M015 | P.S. 015 ROBERTO CLEMENTE | 20052006   | 89.4       | NaN         | 281              | 15   | 36 | 40     | 33     |
| 1 | 01M015 | P.S. 015 ROBERTO CLEMENTE | 20062007   | 89.4       | NaN         | 243              | 15   | 29 | 39     | 38     |
| 2 | 01M015 | P.S. 015 ROBERTO CLEMENTE | 20072008   | 89.4       | NaN         | 261              | 18   | 43 | 39     | 36     |
| 3 | 01M015 | P.S. 015 ROBERTO CLEMENTE | 20082009   | 89.4       | NaN         | 252              | 17   | 37 | 44     | 32     |
| 4 | 01M015 | P.S. 015 ROBERTO CLEMENTE | 20092010   |  _          | 96.5        | 208              | 16   | 40 | 28     | 32     |

In this case, the only column that prevents a given **DBN** from being unique is **schoolyear**. We only want to select rows where schoolyear is **20112012**. This will give us the most recent year of data, and also match our SAT results data.

## 1.7 Condensing the Demographics Data Set

**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>

- Filter **demographics**, only selecting rows in **data["demographics"]** where **schoolyear** is **20112012.**
    - **schoolyear** is actually an integer, so be careful about how you perform your comparison.
- Display the first few rows of **data["demographics"]** to verify that the filtering worked.

In [139]:
# put your code here
demographics = data["demographics"]
demographics = demographics[demographics["schoolyear"] == 20112012]
data["demographics"] = demographics
data["demographics"].head()

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,...,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,P.S. 019 ASHER LEVY,20112012,NaN,61.5,328,32,46,52,54,...,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,PS 020 ANNA SILVER,20112012,NaN,92.5,626,52,102,121,87,...,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3
27,01M034,PS 034 FRANKLIN D ROOSEVELT,20112012,NaN,99.7,401,14,34,38,36,...,90,22.4,275,68.6,8,2.0,204.0,50.9,197.0,49.1
35,01M063,PS 063 WILLIAM MCKINLEY,20112012,NaN,78.9,176,18,20,30,21,...,41,23.3,110,62.5,15,8.5,97.0,55.1,79.0,44.9


## 1.8 Condensing the Graduation Data Set

Finally, we'll need to condense the **graduation** data set. Here are the first few rows:

| _ | Demographic  | DBN    | School Name                           | Cohort   | Total Cohort | Total Grads - n |
|---|--------------|--------|---------------------------------------|----------|--------------|-----------------|
| 0 | Total Cohort | 01M292 | HENRY STREET SCHOOL FOR INTERNATIONAL | 2003     | 5            | s               |
| 1 | Total Cohort | 01M292 | HENRY STREET SCHOOL FOR INTERNATIONAL | 2004     | 55           | 37              |
| 2 | Total Cohort | 01M292 | HENRY STREET SCHOOL FOR INTERNATIONAL | 2005     | 64           | 43              |
| 3 | Total Cohort | 01M292 | HENRY STREET SCHOOL FOR INTERNATIONAL | 2006     | 78           | 43              |
| 4 | Total Cohort | 01M292 | HENRY STREET SCHOOL FOR INTERNATIONAL | 2006 Aug | 78           | 44              |

The **Demographic** and **Cohort** columns are what prevent **DBN** from being unique in the **graduation** data. A **Cohort** appears to refer to the year the data represents, and the **Demographic** appears to refer to a specific demographic group. In this case, we want to pick data from the most recent Cohort available, which is 2006. We also want data from the full cohort, so we'll only pick rows where **Demographic** is **Total Cohort**.

## 1.9 Condensing the Graduation Data Set

**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>

- Filter **graduation**, only selecting rows where the **Cohort** column equals **2006.**
- Filter **graduation**, only selecting rows where the **Demographic** column equals **Total Cohort**.
- Display the first few rows of **data["graduation"]** to verify that everything worked properly.

In [140]:
# put your code here
graduation = data['graduation']
graduation = graduation[graduation['Cohort'] == '2006']
graduation = graduation[graduation['Demographic'] == 'Total Cohort']
data['graduation'] = graduation
data['graduation'].head()

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1,36,46.2,83.7,...,36,46.2,83.7,7,9,16.3,16,20.5,11,14.1
10,Total Cohort,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2006,124,53,42.7,42,33.9,79.2,...,34,27.4,64.2,11,8.9,20.8,46,37.1,20,16.100000000000001
17,Total Cohort,01M450,EAST SIDE COMMUNITY SCHOOL,2006,90,70,77.8,67,74.400000000000006,95.7,...,67,74.400000000000006,95.7,3,3.3,4.3,15,16.7,5,5.6
24,Total Cohort,01M509,MARTA VALLE HIGH SCHOOL,2006,84,47,56,40,47.6,85.1,...,23,27.4,48.9,7,8.300000000000001,14.9,25,29.8,5,6
31,Total Cohort,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHO,2006,193,105,54.4,91,47.2,86.7,...,22,11.4,21,14,7.3,13.3,53,27.5,35,18.100000000000001


## 1.10 Converting AP Test Scores

We're almost ready to combine all of the data sets. The only remaining thing to do is convert the [Advanced Placement (AP)](https://en.wikipedia.org/wiki/Advanced_Placement_exams) test scores from strings to numeric values. High school students take the AP exams before applying to college. There are several AP exams, each corresponding to a school subject. High school students who earn high scores may receive college credit.

AP exams have a 1 to 5 scale; 3 or higher is a passing score. Many high school students take AP exams -- particularly those who attend academically challenging institutions. AP exams are much more rare in schools that lack funding or academic rigor.

It will be interesting to find out whether AP exam scores are correlated with SAT scores across high schools. To determine this, we'll need to convert the AP exam scores in the **ap_2010** data set to numeric values first.

There are three columns we'll need to convert:

- **AP Test Takers** (note that there's a trailing space in the column name)
- **Total Exams Taken**
- **Number of Exams with scores 3 4 or 5**

**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>

- Convert each of the following columns in **ap_2010** to numeric values using the [pandas.to_numeric()](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.to_numeric.html) function with the keyword argument **errors="coerce".**
    - **AP Test Takers**
    - **Total Exams Taken**
    - **Number of Exams with scores 3 4 or 5**
- Display the column types using the **dtypes** attribute.

In [145]:
# put your code here
import pandas as pd

ap_2010 = data['ap_2010']
ap_2010['AP Test Takers '] = pd.to_numeric(ap_2010['AP Test Takers '], errors="coerce")
ap_2010['Total Exams Taken'] = pd.to_numeric(ap_2010['Total Exams Taken'], errors="coerce")
ap_2010['Number of Exams with scores 3 4 or 5'] = pd.to_numeric(ap_2010['Number of Exams with scores 3 4 or 5'], errors="coerce")
data['ap_2010'] = ap_2010
data['ap_2010'].dtypes

DBN                                      object
SchoolName                               object
AP Test Takers                          float64
Total Exams Taken                       float64
Number of Exams with scores 3 4 or 5    float64
dtype: object

## 1.11 Left, Right, Inner, and Outer Joins

Before we merge our data, we'll need to decide on the merge strategy we want to use. We'll be using the pandas [pandas.DataFrame.merge()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) function, which supports four types of joins -- **left**, **right**, **inner**, and **outer**. Each of these join types dictates how pandas combines the rows.

We'll be using the **DBN** column to identify matching rows across data sets. In other words, the values in that column will help us know which row from the first data set to combine with which row in the second data set.

There may be **DBN** values that exist in one data set but not in another. This is partly because the data is from different years. Each data set also has inconsistencies in terms of how it was gathered. Human error (and other types of errors) may also play a role. Therefore, we may not find matches for the **DBN** values in **sat_results** in all of the other data sets, and other data sets may have **DBN** values that don't exist in **sat_results**.

We'll merge two data sets at a time. For example, we'll merge **sat_results** and **hs_directory**, then merge the result with **ap_2010**, then merge the result of that with **class_size**. We'll continue combining data sets in this way until we've merged all of them. Afterwards, we'll have roughly the same number of rows, but each row will have columns from all of the data sets.

The merge strategy we pick will affect the number of rows we end up with. Let's take a look at each strategy.

Let's say we're merging the following two data sets:

<left><img width="300" src="https://drive.google.com/uc?export=view&id=1Vlypix_SIkxCdRS0ABvO4tGiuvFLg321"></left>

With an **inner merge**, we'd only combine rows where the same **DBN** exists in both data sets. We'd end up with this result:

<left><img width="600" src="https://drive.google.com/uc?export=view&id=1LR4c8louX-JAZFYta_Y99FLsGkCf9grr"></left>

With a **left merge**, we'd only use **DBN** values from the dataframe on the "left" of the merge. In this case, **sat_results** is on the left. Some of the DBNs in **sat_results** don't exist in **class_size**, though. The merge will handle this by assiging null values to the columns in **sat_results** that don't have corresponding data in **class_size.**

<left><img width="600" src="https://drive.google.com/uc?export=view&id=1hPoJ5wLECEzz25jrTP5bw9oZ0eerNi2p"></left>

With a **right merge**, we'll only use **DBN** values from the dataframe on the "right" of the merge. In this case, **class_size** is on the right:

<left><img width="600" src="https://drive.google.com/uc?export=view&id=1YYdf4iEMtHYqRBMTEFcfuTyu9zdFlnx7"></left>

With an outer merge, we'll take any DBN values from either sat_results or class_size:

<left><img width="600" src="https://drive.google.com/uc?export=view&id=1sl5wCK3WZ3lTzJm8JUn-bg4MoXl3xSe9"></left>

As you can see, each merge strategy has its advantages. Depending on the strategy we choose, we may preserve rows at the expense of having more missing column data, or minimize missing data at the expense of having fewer rows. Choosing a merge strategy is an important decision; it's worth thinking about your data carefully, and what trade-offs you're willing to make.

Because this project is concerned with determing demographic factors that correlate with SAT score, we'll want to preserve as many rows as possible from **sat_results** while minimizing null values.

This means that we may need to use different merge strategies with different data sets. Some of the data sets have a lot of missing **DBN** values. This makes a **left** join more appropriate, because we don't want to lose too many rows when we merge. If we did an **inner** join, we would lose the data for many high schools.

Some data sets have **DBN** values that are almost identical to those in **sat_results**. Those data sets also have information we need to keep. Most of our analysis would be impossible if a significant number of rows was missing from **demographics**, for example. Therefore, we'll do an inner join to avoid missing data in these columns.

##  1.12 Performing the Left Joins

Both the **ap_2010** and the **graduation** data sets have many missing **DBN** values, so we'll use a left join when we merge the **sat_results** data set with them. Because we're using a **left** join, our final dataframe will have all of the same **DBN** values as the original **sat_results** dataframe.

We'll need to use the pandas **df.merge()** method to merge dataframes. The "left" dataframe is the one we call the method on, and the "right" dataframe is the one we pass into **df.merge()**.

Because we're using the **DBN** column to join the dataframes, we'll need to specify the keyword argument **on="DBN"** when calling **pandas.DataFrame.merge().**

First, we'll assign **data["sat_results"]** to the variable **combined**. Then, we'll merge all of the other dataframes with **combined**. When we're finished, **combined** will have all of the columns from all of the data sets.

**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>

- Use the pandas [pandas.DataFrame.merge()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) method to merge the **ap_2010** data set into **combined.**
    - Make sure to specify **how="left"** as a keyword argument to indicate the correct join type.
    - Make sure to assign the result of the merge operation back to **combined.**
- Use the pandas **df.merge()** method to merge the **graduation** data set into **combined.**
    - Make sure to specify **how="left"** as a keyword argument to get the correct join type.
    - Make sure to assign the result of the merge operation back to **combined.**
- Display the first few rows of **combined** to verify that the correct operations occurred.
- Use the [pandas.DataFrame.shape](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.shape.html) attribute to display the shape of the dataframe and see how many rows now exist.

In [146]:
# put your code here
combined = data['sat_results']
combined = combined.merge(data['ap_2010'], on='DBN', how='left')
combined = combined.merge(data['graduation'], on='DBN', how='left')
combined.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,NaN,NaN,NaN,...,36,46.2,83.7,7,9,16.3,16,20.5,11,14.1
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,...,34,27.4,64.2,11,8.9,20.8,46,37.1,20,16.100000000000001
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0,EAST SIDE COMMUNITY HS,19.0,21.0,...,67,74.400000000000006,95.7,3,3.3,4.3,15,16.7,5,5.6
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0,NaN,NaN,NaN,...,23,27.4,48.9,7,8.300000000000001,14.9,25,29.8,5,6


In [147]:
combined.shape

(479, 33)

## 1.13 Performing the Inner Joins

Now that we've performed the left joins, we still have to merge **class_size**, **demographics**, **survey**, and **hs_directory** into **combined**. Because these files contain information that's more valuable to our analysis and also have fewer missing **DBN** values, we'll use the **inner** join type.

**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>

- Merge **class_size** into **combined**. Then, merge **demographics**, **survey**, and **hs_directory** into **combined** one by one, in that order.
    - Be sure to follow the exact order above.
    - Remember to specify the correct column to join on, as well as the correct join type.
- Display the first few rows of **combined** to verify that the correct operations occurred.
- Call **pandas.DataFrame.shape()** to display the shape of the dataframe to see how many rows now exist.

In [148]:
# put your code here
combined = combined.merge(data['class_size'], on='DBN', how='inner')
combined = combined.merge(data['demographics'], on='DBN', how='inner')
combined = combined.merge(data['survey'], on='DBN', how='inner')
combined = combined.merge(data['hs_directory'], on='DBN', how='inner')
combined.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,lat,lon
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,NaN,NaN,NaN,...,NaN,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,40.713764,-73.985260
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,...,NaN,"200 Monroe Street\nNew York, NY 10002\n(40.712...",3.0,1.0,202.0,1003214.0,1.002590e+09,Lower East Side ...,40.712332,-73.984797
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0,EAST SIDE COMMUNITY HS,19.0,21.0,...,NaN,"420 East 12 Street\nNew York, NY 10009\n(40.72...",3.0,2.0,34.0,1005974.0,1.004390e+09,East Village ...,40.729783,-73.983041
3,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0,NaN,NaN,NaN,...,NaN,"145 Stanton Street\nNew York, NY 10002\n(40.72...",3.0,1.0,3001.0,1004323.0,1.003540e+09,Chinatown ...,40.720569,-73.985673
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522.0,574.0,525.0,1621.0,"NEW EXPLORATIONS SCI,TECH,MATH",255.0,377.0,...,NaN,"111 Columbia Street\nNew York, NY 10002\n(40.7...",3.0,2.0,2201.0,1004070.0,1.003350e+09,Lower East Side ...,40.718725,-73.979426


In [149]:
combined.shape

(363, 165)

##  1.14 Filling in Missing Values

You may have noticed that the inner joins resulted in 116 fewer rows in **sat_results**. This is because pandas couldn't find the **DBN** values that existed in **sat_results** in the other data sets. While this is worth investigating, we're currently looking for high-level correlations, so we don't need to dive into which **DBNs** are missing.

You may also have noticed that we now have many columns with null (**NaN**) values. This is because we chose to do **left** joins, where some columns may not have had data. The data set also had some missing values to begin with. If we hadn't performed a **left** join, all of the rows with missing data would have been lost in the merge process, which wouldn't have left us with many high schools in our data set.

There are several ways to handle missing data, and we'll cover them in more detail later on. For now, we'll just fill in the missing values with the overall mean for the column, like so:

<left><img width="500" src="https://drive.google.com/uc?export=view&id=1OmhXzMuPrGSmyyugGXpmrRlLznDHxOeT"></left>

In the diagram above, the mean of the first column is (1800 + 1600 + 2200 + 2300) / 4, or 1975, and the mean of the second column is (20 + 30 + 30 + 50) / 4, or 32.5. We replace the missing values with the means of their respective columns, which allows us to proceed with analyses that can't handle missing values (like correlations).

We can fill in missing data in pandas using the [pandas.DataFrame.fillna()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) method. This method will replace any missing values in a dataframe with the values we specify. We can compute the mean of every column using the [pandas.DataFrame.mean()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.mean.html) method. If we pass the results of the **df.mean()** method into the **df.fillna()** method, pandas will fill in the missing values in each column with the mean of that column.

Here's an example of how we would accomplish this:

```python
means = df.mean()
df = df.fillna(means)
```

Note that if a column consists entirely of null or **NaN** values, pandas won't be able to fill in the missing values when we use the **df.fillna()** method along with the **df.mean()** method, because there won't be a mean.

We should fill any **NaN** or null values that remain after the initial replacement with the value 0. We can do this by passing 0 into the **df.fillna()** method.

**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>

- Calculate the means of all of the columns in **combined** using the **pandas.DataFrame.mean()** method.
- Fill in any missing values in **combined** with the means of the respective columns using the **pandas.DataFrame.fillna()** method.
- Fill in any remaining missing values in **combined** with 0 using the **df.fillna()** method.
- Display the first few rows of **combined** to verify that the correct operations occurred.

In [156]:
# put your code here
means = combined.mean()
combined = combined.fillna(means)
combined = combined.fillna(0)
combined.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,lat,lon
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,0,129.028846,197.038462,...,0,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,40.713764,-73.985260
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.000000,49.000000,...,0,"200 Monroe Street\nNew York, NY 10002\n(40.712...",3.0,1.0,202.0,1003214.0,1.002590e+09,Lower East Side ...,40.712332,-73.984797
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0,EAST SIDE COMMUNITY HS,19.000000,21.000000,...,0,"420 East 12 Street\nNew York, NY 10009\n(40.72...",3.0,2.0,34.0,1005974.0,1.004390e+09,East Village ...,40.729783,-73.983041
3,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0,0,129.028846,197.038462,...,0,"145 Stanton Street\nNew York, NY 10002\n(40.72...",3.0,1.0,3001.0,1004323.0,1.003540e+09,Chinatown ...,40.720569,-73.985673
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522.0,574.0,525.0,1621.0,"NEW EXPLORATIONS SCI,TECH,MATH",255.000000,377.000000,...,0,"111 Columbia Street\nNew York, NY 10002\n(40.7...",3.0,2.0,2201.0,1004070.0,1.003350e+09,Lower East Side ...,40.718725,-73.979426


## 1.15 Adding a School District Column for Mapping

We've finished cleaning and combining our data! We now have a clean data set on which we can base our analysis. Mapping the statistics out on a school district level might be an interesting way to analyze them. Adding a column to the data set that specifies the school district will help us accomplish this.

The school district is just the first two characters of the **DBN**. We can apply a function over the **DBN** column of **combined** that pulls out the first two letters.

For example, we can use indexing to extract the first few characters of a string, like this:

```python
name = "Sinbad"
print(name[0:2])
```

**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>

- Write a function that extracts the first two characters of a string and returns them.
- Apply the function to the **DBN** column of **combined**, and assign the result to the **school_dist** column of **combined**.
- Display the first few items in the **school_dist** column of **combined** to verify the results.


In [0]:
# put your code here
def school_district(string):
  return string[0:2]

In [158]:
combined['school_dist'] = combined['DBN'].apply(school_district)
combined["school_dist"].head()

0    01
1    01
2    01
3    01
4    01
Name: school_dist, dtype: object

## 1.16 Next Steps

We now have a clean data set we can analyze! We've done a lot in this mission. We've gone from having several messy sources to one clean, combined, data set that's ready for analysis.

Along the way, we've learned about:

- How to handle missing values
- Different types of merges
- How to condense data sets
- How to compute averages across dataframes

Data scientists rarely start out with tidy data sets, which makes cleaning and combining them one of the most critical skills any data professional can learn.
